<a href="https://colab.research.google.com/github/athirajoshy/firstrepository/blob/master/face_mask_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.optimizers import Adam

In [ ]:
from tensorflow.keras.layers import Input,Flatten,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
image_size=[224,224]
train_data_path='/content/drive/MyDrive/dataset_train'
valid_data_path='/content/drive/MyDrive/test_dataset_mask'

In [ ]:
inception=InceptionV3(input_shape=image_size+[3],weights='imagenet',include_top=False)

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
for i in inception.layers:
  i.trainable=False

In [ ]:
folders = glob('/content/drive/My Drive/dataset_train/*')
len(folders)

2

In [ ]:
x = Flatten()(inception.output)#output from inception is flattened

In [ ]:
prediction = Dense(len(folders), activation='sigmoid')(x)

# create a model object
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
model.compile(
  loss='sparse_categorical_crossentropy',
  optimizer='Adam',
  metrics=['accuracy']
)

In [ ]:
#data augmentation
training_data=ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


In [ ]:
training_set = training_data.flow_from_directory('/content/drive/My Drive/dataset_train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 3834 images belonging to 2 classes.


In [ ]:
training_set.class_indices

{'with_mask': 0, 'without_mask': 1}

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
testing_data = test_datagen.flow_from_directory('/content/drive/My Drive/test_dataset_mask',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 452 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=testing_data,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(testing_data)
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
 31/120 [======>.......................] - ETA: 17:08 - loss: 1.9146 - accuracy: 0.8151

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


120/120 [==============================] - ETA: 0s - loss: 0.8974 - accuracy: 0.9173 

/usr/local/lib/python3.7/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 4 bytes but only got 0. 
  warnings.warn(str(msg))


120/120 [==============================] - 1571s 13s/step - loss: 0.8929 - accuracy: 0.9177 - val_loss: 0.9199 - val_accuracy: 0.9403
Epoch 2/10
120/120 [==============================] - 496s 4s/step - loss: 0.0596 - accuracy: 0.9947 - val_loss: 0.7259 - val_accuracy: 0.9358
Epoch 3/10
120/120 [==============================] - 499s 4s/step - loss: 0.0427 - accuracy: 0.9957 - val_loss: 2.0644 - val_accuracy: 0.8894
Epoch 4/10
120/120 [==============================] - 501s 4s/step - loss: 0.0482 - accuracy: 0.9953 - val_loss: 1.1975 - val_accuracy: 0.9159
Epoch 5/10
120/120 [==============================] - 501s 4s/step - loss: 0.0668 - accuracy: 0.9954 - val_loss: 2.0947 - val_accuracy: 0.9093
Epoch 6/10
120/120 [==============================] - 497s 4s/step - loss: 0.0553 - accuracy: 0.9958 - val_loss: 1.6303 - val_accuracy: 0.9226
Epoch 7/10
120/120 [==============================] - 498s 4s/step - loss: 0.0321 - accuracy: 0.9973 - val_loss: 2.1423 - val_accuracy: 0.9137
Epoch 8/

In [ ]:
from tensorflow.keras.models import load_model

model.save('/content/drive/MyDrive/models/model_mask_inception.h5')